# Deep learning A-Z : Building a CNN

This notebook is my response to the second homework of the course called *Deep Learning A-Z™: Hands-On Artificial Neural Networks* accessible here : https://www.udemy.com/deeplearning/

In this notebook, we are going to build an CNN using keras and by following instructions given on the course. This neurals network will predict, for an image, if this a cat or a dog image. We are going to train our ann with a lot of cat and dog images and after that, we are going to make predictions.

### Imports

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Data preprocessing

There is no data preprocessing with images because independant variables are our pixels and data are already divided into train and test set in the structure of our projet. However, we will need to standardize our data before fiting our model

### 2. Building the CNN

In [2]:
# Initializing our CNN
classifier = Sequential()

As we seen it in the course, we are going to ad a layer for each step for processing images before passing it to a classic ann (full connection):

![title](images/steps_cnn.png)


In [3]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3,3), input_shape=(64, 64, 3), activation='relu'))

- *filters* is the number of feature detector we will use (common practice = start with 32). C
- *kernel_size* is the size of the feature detectors.
- *input_shape* used to fix the size of our images, which have by default differents sizes. Third number corresponds to number of channels (color or B&W). 6' is enough here because we work on a CPU, but an be increase for a better accuracy on GPU.
- *activation* is used, as seen in the course, to remove non-linearity on our ouput convolutioned images. We will use relu.

In [4]:
# Step 2 - Max Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

- *pool_size* corresponds to the size of the max pooling subtable used to reduce size of feature map. In general, 2x2 matrix is enough.

In [5]:
# Step 3 - Flattening
classifier.add(Flatten())

In [6]:
# Step 4 - Full connection, add hidden layer (fully connected layer)
classifier.add(Dense(units=128, activation='relu'))

In [7]:
# Step 4 - Full connection - add output layer
classifier.add(Dense(units=1, activation='sigmoid'))

In [8]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

We are going to use an images augmentation process using keras to reduce overfitting which can appears because of our few number of images (only 8000 for training). Image augmentation allows us to enrich our dataset without adding new images. It will, for each images, create a batch of images with random transformation on it (rotating, flipping, shifting …) so it will provide a lot more material to train our CNN.

In [9]:
path_train = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 
                          'ressources/Convolutional_Neural_Networks/dataset/training_set')
path_test = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 
                          'ressources/Convolutional_Neural_Networks/dataset/test_set')

In [11]:
# code for image augmentation found here : https://keras.io/preprocessing/image/

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True) # Apply image augmentation on train images

test_datagen = ImageDataGenerator(rescale=1./255) # Do not apply transformation on test images

# target size is the same as choosen in our cnn architecture
training_set = train_datagen.flow_from_directory(path_train, target_size=(64, 64),batch_size=32, 
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory(path_test, target_size=(64, 64), batch_size=32, 
                                            class_mode='binary')

# fit model
classifier.fit_generator(training_set, 
                         steps_per_epoch=8000, # number of images in training_set
                         epochs=1, # One epoch because my computer is too slow (need to be increased)
                         validation_data=test_set, 
                         validation_steps=2000) # number of images in training_set

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
8000/8000 [==============================] - 3360s 420ms/step - loss: 0.2449 - acc: 0.8966 - val_loss: 0.9785 - val_acc: 0.7531


Accuracy is not as good as we would expect on test set ... How can we increase it ? By building a deeper network ! We have the choice between adding another convolutionnal layer or adding another fully connected layer. In fact, adding  convolutional yaer is often the best solution. Let's do it:

In [12]:
deeper_classifier = Sequential()
deeper_classifier.add(Conv2D(32, (3,3), input_shape=(64, 64, 3), activation='relu'))
deeper_classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add a second convolutionnal layer, input_shape is not necessary because there is other layer before
deeper_classifier.add(Conv2D(32, (3,3), activation='relu'))
deeper_classifier.add(MaxPooling2D(pool_size=(2,2)))

deeper_classifier.add(Flatten())
deeper_classifier.add(Dense(units=128, activation='relu'))
deeper_classifier.add(Dense(units=1, activation='sigmoid'))
deeper_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

deeper_classifier.fit_generator(training_set, 
                         steps_per_epoch=8000, # number of images in training_set
                         epochs=2, 
                         validation_data=test_set, 
                         validation_steps=2000) # number of images in training_set

Epoch 1/2
8000/8000 [==============================] - 3716s 464ms/step - loss: 0.3408 - acc: 0.8381 - val_loss: 0.6987 - val_acc: 0.7833
Epoch 2/2
8000/8000 [==============================] - 4392s 549ms/step - loss: 0.0721 - acc: 0.9737 - val_loss: 1.0910 - val_acc: 0.7865


In [15]:
# save model
deeper_classifier.save("cnn.h5")

# reload model
from keras.models import load_model
#deeper_classifier = load_model('cnn.h5')

### 3. Homework

Predict the class of two pictures:

In [72]:
path_prediction = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 
                          'ressources/Convolutional_Neural_Networks/dataset/single_prediction/')

image_1 = image.load_img(path_prediction+'cat_or_dog_1.jpg', target_size=(64, 64))
image_1 = image.img_to_array(image_1)
image_1 = np.expand_dims(image_1, axis=0)

image_2 = image.load_img(path_prediction+'cat_or_dog_2.jpg', target_size=(64, 64))
image_2 = image.img_to_array(image_2)
image_2 = np.expand_dims(image_2, axis=0)

In [89]:
prediction = deeper_classifier.predict(image_1)
if prediction[0][0] == training_set.class_indices['dogs']:
    prediction='dog'
else:
    prediction='cat'
print(prediction)

dog
